In [ ]:
import numpy as np
import joblib
import regex as re
import pandas as pd
import time
from datetime import date
import requests
from openpyxl.chart import LineChart, Reference
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Font
from openpyxl.utils import get_column_letter


In [ ]:
session=requests.Session()
session.get("https://portal.amfiindia.com", timeout=5)
session.get("https://api.mfapi.in", timeout=5)


In [ ]:
invested_details=joblib.load('Folio.pkl')
print(invested_details)
invested_details

In [ ]:
df=pd.read_csv('id.csv')


In [ ]:
print(df)

In [ ]:
pattern=re.compile(r'([0-9]+)-([0-9]+)-([0-9]+)')

In [ ]:
def datemaker(date_str):

    match=pattern.match(date_str)
    day=int(match.group(1))
    month=int(match.group(2))
    year=int(match.group(3))
    return date(year,month,day)

In [ ]:
name_to_id = dict(zip(df["Name"], df["ID"]))

In [ ]:
data = {}
from functools import lru_cache
@lru_cache(maxsize=None)
def fetch_nav(comp):
    global data
    myid = name_to_id[comp]
    url = "https://portal.amfiindia.com/spages/NAVAll.txt"

    if not data:
        lines = session.get(url, timeout=10).text.splitlines()
        for line in lines:
            parts = line.split(";")
            data.setdefault(parts[0], parts)  
    else:
        print("Using cached NAV data")

    if myid not in data:
        raise ValueError(f"NAV not found for {comp} with id {myid}")

    latest = data[myid]
    nav = float(latest[-2])
    date_str = latest[-1]

    month_map = {
        "Jan":"01","Feb":"02","Mar":"03","Apr":"04",
        "May":"05","Jun":"06","Jul":"07","Aug":"08",
        "Sep":"09","Oct":"10","Nov":"11","Dec":"12"
    }

    d, m, y = date_str.split("-")
    return nav, f"{d}-{month_map[m]}-{y}"


In [ ]:
_nav_cache = {}

def fetch_nav_on_date(comp, target_date):
    if comp not in _nav_cache:
        myid = name_to_id[comp]
        url = f"https://api.mfapi.in/mf/{myid}"
        raw = session.get(url, timeout=10).json()["data"]

        _nav_cache[comp] = {
            entry["date"]: float(entry["nav"])
            for entry in raw
        }

    return _nav_cache[comp].get(target_date, 0)


In [ ]:
def unit_tracker(item):
    total_units = 0
    for entry in invested_details[item]:
        date_str, amount = entry
        if amount < 0:
            amount=amount*(1-0.005/100)  # assuming 0.005% exit load
        amount=-amount
        nav = fetch_nav_on_date(item, date_str)
        if nav is None:
            print(f"NAV not found for {item} on {date_str}")
            continue
        units = amount / nav
        total_units += units
    return total_units

In [ ]:
#formula and code of this cell from CHatGPT
import scipy.optimize

def xnpv(rate, values, dates):
    if rate <= -1.0:
        return float('inf')
    d0 = dates[0]
    return sum(
        v / (1.0 + rate) ** ((d - d0).days / 365.0)
        for v, d in zip(values, dates)
    )


def xirr(values, dates):

    if len(values) == 2:
        v0, v1 = values
        if v0 >= 0 or v1 <= 0:
            raise ValueError("Invalid cashflow signs for XIRR")

        days = (dates[1] - dates[0]).days / 365.0
        return (v1 / -v0) ** (1 / days) - 1


    def f(r):
        return xnpv(r, values, dates)

    guess = 0.12

    try:
        return scipy.optimize.newton(
            f,
            guess,
            tol=1e-6,
            maxiter=25
        )
    except (RuntimeError, OverflowError):
        pass

    return scipy.optimize.brentq(
        f,
        -0.999,     
        10.0,        
        maxiter=50
    )


In [ ]:
def xirrcalc(item):
    dates = []
    values = []

    for date_str, amount in invested_details[item]:
        dates.append(datemaker(date_str))
        values.append(amount)   

    nav, nav_date = fetch_nav(item)
    last_nav_date=nav_date
    lastpossibledate=date.today()
    for i in range(1,(lastpossibledate-datemaker(nav_date)).days+1):
        checkdate=datemaker(nav_date)+timedelta(days=i)
        checkdate_str=checkdate.strftime("%d-%m-%Y")
        if fetch_nav_on_date(item,checkdate_str)!=0:
            nav_date=checkdate_str
            nav=fetch_nav_on_date(item,checkdate_str)
            print(f'Using NAV from date {nav_date} for fund {item} using MFapi as amfi had {last_nav_date} ')
    units_left = unit_tracker(item)
    dates.append(datemaker(nav_date))
    values.append(units_left * nav)   
    print(dates)
    print(values)
    z=time.time()
    a=xirr(values, dates) * 100
    print(f'{time.time()-z} for XIRR calculation') 
    return (a, units_left*nav,units_left,nav,nav_date)  

In [ ]:
from datetime import timedelta
def portfolio(fundname,end):
    portfolio_sum=[]
    x=invested_details[fundname]
    x=list(sorted(x,key=lambda x:datemaker(x[0])))
    startdate=datemaker(x[0][0])
    enddate=end
    totalinvestamnt={}
    currinvestment={}
    tot=0
    totunits=0
    g=0

    for i in range((enddate-startdate).days+1):
        currdate=startdate+timedelta(days=i)
        currdate_str=currdate.strftime("%d-%m-%Y")
        while True:
            if g<len(x) and datemaker(x[g][0])<=currdate:
                tot-=x[g][1]
                z=fetch_nav_on_date(fundname,x[g][0])
                if z==0:
                    print(f'NAV not found for {fundname} on date {x[g][0]} and it was in purchase history')
                totunits+=(-x[g][1]*(1-0.005/100) if x[g][1]<0 else -x[g][1])/z 
                g+=1
            else:
                break
        totalinvestamnt[currdate_str]=tot
        currinvestment[currdate_str]=totunits*fetch_nav_on_date(fundname,currdate_str)
    lastinv=0
    lastdate=None
    for a in totalinvestamnt:
        if currinvestment[a]!=0:
            lastdate=a
        lastinv=currinvestment[a] if currinvestment[a]!=0 else lastinv
        portfolio_sum.append([a,totalinvestamnt[a],currinvestment[a] if currinvestment[a]!=0 else lastinv,currinvestment[a]-totalinvestamnt[a] if currinvestment[a]!=0 else lastinv - totalinvestamnt[a]])
    df=pd.DataFrame(portfolio_sum,columns=['Date','Total Invested Amount','Current Investment Value','Profit/Loss'])
    return df

In [ ]:
#ChatGPT on

def enhance(name):
    wb = load_workbook(name)  # or your file name
    ws = wb.active

    # --- CREATE LINE CHART ---
    chart = LineChart()
    chart.title = "Date vs Profit / Loss"
    chart.y_axis.title = "Profit / Loss (₹)"
    chart.x_axis.title = "Date"

    # --- DATA RANGE ---
    # Profit/Loss column (D = 4 in your table)
    data = Reference(
        ws,
        min_col=4,        # Profit/Loss
        min_row=1,
        max_row=ws.max_row
    )

    # Date column (A = 1)
    cats = Reference(
        ws,
        min_col=1,        # Date
        min_row=2,
        max_row=ws.max_row
    )

    chart.add_data(data, titles_from_data=True)
    chart.set_categories(cats)

    # --- ENABLE SMOOTH (CUBIC SPLINE) ---
    for series in chart.series:
        series.smooth = False   # THIS is the key line
    chart.width = 28     # inches
    chart.height = 18    # inches


    # --- INSERT CHART ---
    ws.add_chart(chart, "F2")

    wb.save(name)

#ChatGPT off

To add more investments

In [ ]:
def add(name,date,amount):
    if name in invested_details:
        invested_details[name].append((date,-amount))
    else:
        invested_details[name]=[(date,-amount)]
    return f'Added {amount} to fund {name} on date {date}'

To sell it

In [ ]:
def sell(name,date,amount):
    if name in invested_details:
        invested_details[name].append((date,amount))
    else:
        invested_details[name]=[(date,amount)]
    return f'Sold {amount} to fund {name} on date {date}'

Please add using dashes

Final main code

In [ ]:
xirrdict=[]

net_dates = []
net_values = []
total_currval = 0
import time
for item in invested_details.keys():
    z=time.time()
    ahh = xirrcalc(item)
    print("XIRRCALC done")
    print(time.time()-z)
    
    currval = ahh[1]
    xirrval = ahh[0]
    tot = 0
    for date_str, amount in invested_details[item]:
        tot -= amount
        net_dates.append(datemaker(date_str))   
        net_values.append(amount)
    invested_total = tot
    returns = currval - invested_total
    avgcost = invested_total / ahh[2] if ahh[2] != 0 else 0

    total_currval += currval   

    xirrdict.append(
        (ahh[4], item, invested_total, currval, returns, avgcost, ahh[3], xirrval)
    )
    
net_dates.append(max(datemaker(x[0]) for x in xirrdict))
net_values.append(total_currval)

net_xirr = xirr(net_values, net_dates)  
net_xirr_percent = net_xirr * 100
df_tx = pd.DataFrame(
    xirrdict,
    columns=[
        "Date","Fund Name","Invested Total","Current Value",
        "Returns","Average Cost per Unit","Current Price per unit","XIRR"
    ]
)

total_row = {
    "Date": "",
    "Fund Name": "TOTAL",
    "Invested Total": df_tx["Invested Total"].sum(),
    "Current Value": df_tx["Current Value"].sum(),
    "Returns": df_tx["Returns"].sum(),
    "Average Cost per Unit": "",
    "Current Price per unit": "",
    "XIRR": net_xirr_percent
}

df_tx = pd.concat([df_tx, pd.DataFrame([total_row])], ignore_index=True)
df_tx.to_excel("FolioAnalysis.xlsx", index=False)


In [ ]:
#--ChatGPT Addition Start--#


wb = load_workbook("FolioAnalysis.xlsx")
ws = wb.active

# colors
green = PatternFill(fill_type="solid", start_color="C6EFCE", end_color="C6EFCE")
red   = PatternFill(fill_type="solid", start_color="FFC7CE", end_color="FFC7CE")

# map headers → column index
headers = {cell.value: idx+1 for idx, cell in enumerate(ws[1])}

returns_col = headers["Returns"]
xirr_col = headers["XIRR"]

# color profit / loss cells
for r in range(2, ws.max_row + 1):
    ret = ws.cell(row=r, column=returns_col)
    xir = ws.cell(row=r, column=xirr_col)

    if ret.value is not None:
        ret.fill = green if ret.value >= 0 else red

    if xir.value is not None:
        xir.fill = green if xir.value >= 0 else red

# ₹ formatting
rupee_cols = [
    "Invested Total",
    "Current Value",
    "Returns",
    "Average Cost per Unit",
    "Current Price per unit"
]

for col_name in rupee_cols:
    c = headers[col_name]
    for r in range(2, ws.max_row + 1):
        ws.cell(row=r, column=c).number_format = "₹#,##0.00"

# XIRR percent format (NO math change)
for r in range(2, ws.max_row + 1):
    ws.cell(row=r, column=xirr_col).number_format = '0.00"%"'


# bold header + freeze
for cell in ws[1]:
    cell.font = Font(bold=True)

ws.freeze_panes = "A2"

# auto column width
for col in ws.columns:
    max_len = max(len(str(cell.value)) if cell.value else 0 for cell in col)
    ws.column_dimensions[get_column_letter(col[0].column)].width = max_len + 2

wb.save("FolioAnalysis.xlsx")
#--ChatGPT Addition End--#

In [ ]:
b = None
for item in invested_details.keys():
    a = portfolio(item,end=(max(datemaker(x[0]) for x in xirrdict)))
    a = a.set_index("Date")

    if b is None:
        b = a
    else:
        b = b.add(a, fill_value=0)
    a.reset_index().to_excel(f'Portfolio_{item}.xlsx', index=False)
    enhance(f'Portfolio_{item}.xlsx')
    print(f'Processed {item}')
b.reset_index().to_excel('Portfolio_Total.xlsx', index=False)
enhance('Portfolio_Total.xlsx')
print('Processed Total Portfolio')


Save

In [ ]:
joblib.dump(invested_details,'Folio.pkl')